<a href="https://colab.research.google.com/github/KwonYN/TIL/blob/master/PYTHON/TENSORFLOW2/05_05_googlenet_7_kyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import numpy as np
import cv2 

from tensorflow.keras.datasets import cifar10
#from tensorflow.keras.np_utils import np_utils
from tensorflow.keras.optimizers import SGD,Adam

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Inception 모듈 정의
def inception_module(x, x1_oc=64, x1x3_oc=64, x3_oc=128, x1x5_oc=16, x5_oc=32, mp_oc=32):
    x_1 = layers.Conv2D(x1_oc, 1, padding='same')(x)
    x_2 = layers.Conv2D(x1x3_oc, 1, padding='same')(x)
    x_2 = layers.Conv2D(x3_oc, 3, padding='same')(x_2)
    x_3 = layers.Conv2D(x1x5_oc, 1, padding='same')(x)
    x_3 = layers.Conv2D(x5_oc, 5, padding='same')(x_3)
    x_4 = layers.MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    x_4 = layers.Conv2D(mp_oc, 1, padding='same')(x_4)
    return layers.concatenate([x_1, x_2, x_3, x_4])

In [0]:
img_shape = (224, 224, 3)

In [0]:
input_ = layers.Input(shape=img_shape)
x = layers.Conv2D(64, 7, strides=2, padding='same')(input_)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 1, strides=1)(x)
x = layers.Conv2D(192, 3, strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, x1_oc=64, x1x3_oc=64, x3_oc=128, x1x5_oc=16, x5_oc=32, mp_oc=32)
x = inception_module(x, x1_oc=128, x1x3_oc=128, x3_oc=192, x1x5_oc=32, x5_oc=96, mp_oc=64)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, x1_oc=192, x1x3_oc=96, x3_oc=208, x1x5_oc=16, x5_oc=48, mp_oc=64)
x = inception_module(x, x1_oc=160, x1x3_oc=112, x3_oc=224, x1x5_oc=24, x5_oc=64, mp_oc=64)
x = inception_module(x, x1_oc=128, x1x3_oc=128, x3_oc=256, x1x5_oc=24, x5_oc=64, mp_oc=64)
x = inception_module(x, x1_oc=112, x1x3_oc=144, x3_oc=288, x1x5_oc=32, x5_oc=64, mp_oc=64)
x = inception_module(x, x1_oc=256, x1x3_oc=160, x3_oc=320, x1x5_oc=32, x5_oc=128, mp_oc=128)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
x = inception_module(x, x1_oc=256, x1x3_oc=160, x3_oc=320, x1x5_oc=32, x5_oc=128, mp_oc=128)
x = inception_module(x, x1_oc=384, x1x3_oc=192, x3_oc=384, x1x5_oc=48, x5_oc=128, mp_oc=128)
x = layers.AveragePooling2D(pool_size=(7, 7), strides=1)(x)
x = layers.BatchNormalization()(x)
#x = layers.Dropout(0.4)(x)
#x = layers.Dense(1000)(x)
x = layers.Dense(10)(x)
output = layers.Activation('softmax')(x)

googlenet = Model(input_, output)

In [0]:
googlenet.summary()
print("전체 파라미터 수 : {}".format(sum([arr.flatten().shape[0] for arr in googlenet.get_weights()])))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 56, 64)   256         max_pooling2d[0][0]              
______________________________________________________________________________________________

### cv2 로 하니까 메모리에 모두 쌓여서 돌릴수가 없어 

In [0]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols,n_image=100):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[0:n_image,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[0:n_image,:,:,:]])

    Y_train = Y_train[0:n_image,:]
    Y_valid = Y_valid[0:n_image,:]
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [0]:
X_train, y_train, X_test, y_test = load_cifar10_data(224, 224,n_image=5000)

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
epochs = 25
#lrate = 0.01
#decay = lrate/epochs
#sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
googlenet.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
googlenet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64,validation_freq=5)

Train on 5000 samples, validate on 5000 samples
Epoch 1/25
5000/5000 [==============================] - 14s 3ms/sample - loss: 2.4358 - accuracy: 0.1022
Epoch 2/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 2.2337 - accuracy: 0.1005
Epoch 3/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 2.3763 - accuracy: 0.1010
Epoch 4/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 2.5067 - accuracy: 0.1015
Epoch 5/25
5000/5000 [==============================] - 15s 3ms/sample - loss: 2.0430 - accuracy: 0.0993 - val_loss: 2.2885 - val_accuracy: 0.1012
Epoch 6/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 2.0085 - accuracy: 0.1019
Epoch 7/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 1.9732 - accuracy: 0.1018
Epoch 8/25
5000/5000 [==============================] - 10s 2ms/sample - loss: 1.7670 - accuracy: 0.1023
Epoch 9/25
5000/5000 [==============================] - 10s 2ms/sample